In [3]:
# import dependecies 
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import datetime

#Config should contain database username as username and database password as password
import config

In [4]:
# connect to SQL database 

protocol = 'postgresql'
username = config.username
password = config.password
host = 'localhost'
port = 5432
database_name = config.database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)
conn = engine.connect()

In [5]:
#Fetch station data
stationData = pd.read_csv("./Resources/stationData.csv")
stationData.head()

,WMO_NUM,station_number,station_name,LATITUDE,LONGITUDE,STN_HT,AVIATION_ID,REGION,GridPt Lat,GridPt Lon,MSAS elevation,Distance from GridPt,Roughness,Distance from coast,Category,forecast_district,sa_special
0,94648,23000,ADELAIDE (WEST TERRACE / NGAYIRDAPIRA),-34.9257,138.5832,29.32,ADWT,SA,NaN,NaN,NaN,NaN,NaN,NaN,coast,SA_PW001,sag
1,94489,38076,WINDORAH AP,-25.4117,142.6647,132.16,YWDH,QLD,NaN,NaN,NaN,NaN,NaN,NaN,flat_inland,QLD_PW011,NaN
2,94795,9281,MILLENDON (SWAN VALLEY),-31.8108,116.0225,16.00,SWVA,WA,NaN,NaN,NaN,NaN,NaN,NaN,mountains2,WA_PW009,NaN
3,99218,32194,COWLEY BEACH,-17.6904,146.1126,17.00,CBTA,QLD,NaN,NaN,NaN,NaN,NaN,NaN,coast,QLD_PW004,NaN
4,94794,51164,GIRILAMBONE (OKEH),-31.0822,146.9294,178.00,NDR2,NSW,NaN,NaN,NaN,NaN,NaN,NaN,flat_inland,NSW_PW012,NaN


In [6]:
#Make sure to match schema
stationData_sql = pd.DataFrame()
stationData_sql["station_number"] = stationData["station_number"]
stationData_sql["station_name"] = stationData["station_name"]
stationData_sql["lat"] = stationData["LATITUDE"]
stationData_sql["lon"] = stationData["LONGITUDE"]
stationData_sql["height"] = stationData["STN_HT"]
stationData_sql["region"] = stationData["REGION"]
stationData_sql.to_sql("station_data", conn,if_exists='append',index=False)



575

In [7]:
#Get file list for fcst and obs tables
fcst_files = os.listdir("./Resources/fcst")
print(fcst_files)
fcst_data = pd.DataFrame()
for file in fcst_files:
    date = datetime.datetime.strptime(file[-12:-4],"%Y%m%d")
    tempdf = pd.read_csv("./Resources/fcst/" + file)
    first_seconds = tempdf["base_time"][0]
    tempdf_sql = pd.DataFrame()
    tempdf_sql["station_number"] = tempdf["station_number"]
    tempdf_sql["area_code"] = tempdf["area_code"]
    tempdf_sql["valid_start"]  = (tempdf["valid_start"] - first_seconds)/(60*60)
    tempdf_sql["valid_end"]  = (tempdf["valid_end"] - first_seconds)/(60*60)
    tempdf_sql["temperature"]  = (tempdf["valid_end"] - first_seconds)/(60*60)
    tempdf_sql["date"] = date
    tempdf_sql.to_sql("fcst",conn,if_exists='append',index=False)

['Op_Official_0000_20160501.csv', 'Op_Official_0000_20160601.csv', 'Op_Official_0000_20160701.csv', 'Op_Official_0000_20160801.csv', 'Op_Official_0000_20160901.csv', 'Op_Official_0000_20161001.csv', 'Op_Official_0000_20161101.csv', 'Op_Official_0000_20161201.csv', 'Op_Official_0000_20170101.csv', 'Op_Official_0000_20170201.csv', 'Op_Official_0000_20170301.csv', 'Op_Official_0000_20170401.csv']


In [8]:
#Get file list for fcst and obs tables
obs_files = os.listdir("./Resources/obs")
print(fcst_files)
fcst_data = pd.DataFrame()
for file in obs_files:
    date = datetime.datetime.strptime(file[-12:-4],"%Y%m%d")
    tempdf = pd.read_csv("./Resources/obs/" + file)
    first_seconds = tempdf["valid_start"][0]
    tempdf_sql = pd.DataFrame()
    tempdf_sql["station_number"] = tempdf["station_number"]
    tempdf_sql["area_code"] = tempdf["area_code"]
    tempdf_sql["valid_start"]  = (tempdf["valid_start"] - first_seconds)/(60*60)
    tempdf_sql["valid_end"]  = (tempdf["valid_end"] - first_seconds)/(60*60)
    tempdf_sql["temperature"]  = (tempdf["valid_end"] - first_seconds)/(60*60)
    tempdf_sql["date"] = date
    tempdf_sql.to_sql("obs",conn,if_exists='append',index=False)

['Op_Official_0000_20160501.csv', 'Op_Official_0000_20160601.csv', 'Op_Official_0000_20160701.csv', 'Op_Official_0000_20160801.csv', 'Op_Official_0000_20160901.csv', 'Op_Official_0000_20161001.csv', 'Op_Official_0000_20161101.csv', 'Op_Official_0000_20161201.csv', 'Op_Official_0000_20170101.csv', 'Op_Official_0000_20170201.csv', 'Op_Official_0000_20170301.csv', 'Op_Official_0000_20170401.csv']
